In [1]:
!pip install openai icalendar dateparser pytz


   ------------- -------------------------- 1/3 [icalendar]
   ------------- -------------------------- 1/3 [icalendar]
   -------------------------- ------------- 2/3 [dateparser]
   -------------------------- ------------- 2/3 [dateparser]
   -------------------------- ------------- 2/3 [dateparser]
   ---------------------------------------- 3/3 [dateparser]



In [ ]:
"""
text_to_ics_envfile.py
Reads the OpenAI API key from a .env file (e.g., temp.env),
then uses the LLM to generate an .ics calendar file from free text.
"""

import os
import json
from datetime import datetime, timedelta
import pytz
import dateparser
from icalendar import Calendar, Event, vText
from openai import OpenAI
from dotenv import load_dotenv

# --- Load environment file ---
load_dotenv("temp.env")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise EnvironmentError("❌ OPENAI_API_KEY not found in temp.env or environment variables.")

client = OpenAI(api_key=api_key)

DEFAULT_TZ = "UTC"

def parse_event_with_llm(text):
    """
    Extract structured calendar info using OpenAI.
    """
    system_prompt = (
    "You are an assistant that extracts calendar event data from text. "
    "Respond ONLY with a valid JSON object (no markdown, no explanations). "
    "Include these fields: title, start_time, end_time, location, description. "
    "Use ISO 8601 datetime format. "
    "If end_time is missing, set it one hour after start_time."
)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text},
        ],
        temperature=0,
    )

    content = response.choices[0].message.content.strip()
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        raise ValueError(f"Model output was not valid JSON:\n{content}")

def create_ics(event_data, output_path="event.ics"):
    """
    Build and save an .ics file.
    """
    cal = Calendar()
    cal.add('prodid', '-//OpenAI Calendar Agent//')
    cal.add('version', '2.0')

    event = Event()
    event.add('summary', event_data['title'])
    event.add('description', event_data.get('description', ''))
    if event_data.get('location'):
        event.add('location', vText(event_data['location']))

    tz = pytz.timezone(DEFAULT_TZ)
    start = dateparser.parse(event_data['start_time'])
    if not start:
        raise ValueError("Unable to parse start_time")
    end = dateparser.parse(event_data.get('end_time'))
    if not end:
        end = start + timedelta(hours=1)

    event.add('dtstart', start.astimezone(tz))
    event.add('dtend', end.astimezone(tz))
    event.add('dtstamp', datetime.now(tz))
    event['uid'] = f"{datetime.now().timestamp()}@llm-agent"

    cal.add_component(event)

    with open(output_path, "wb") as f:
        f.write(cal.to_ical())

    return output_path

def text_to_ics(text, output_path="event.ics"):
    event_data = parse_event_with_llm(text)
    file_path = create_ics(event_data, output_path)
    return file_path, event_data




In [4]:
if __name__ == "__main__":
    user_text = input("Enter event text:\n> ")
    ics_path, parsed = text_to_ics(user_text)
    print("\n✅ Parsed Event:")
    print(json.dumps(parsed, indent=2))
    print(f"\n📅 ICS file saved as: {ics_path}")

Enter event text:
>  Meeting with product team next Tuesday at 10am at HQ, discuss launch roadmap.



✅ Parsed Event:
{
  "title": "Meeting with product team",
  "start_time": "2023-10-31T10:00:00",
  "end_time": "2023-10-31T11:00:00",
  "location": "HQ",
  "description": "Discuss launch roadmap."
}

📅 ICS file saved as: event.ics


C:\Users\chock\AppData\Local\Temp\ipykernel_31104\3433557722.py:69: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  start = dateparser.parse(event_data['start_time'])
C:\Users\chock\AppData\Local\Temp\ipykernel_31104\3433557722.py:72: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  end = dateparser.parse(event_data.get('end_time'))
